<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 8: Fitting Neutrino Data</h1>


<a name='section_8_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_8_1">L8.1 Neutrino Oscillations</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_8_1">L8.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_8_2">L8.2 Loading the Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_8_2">L8.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_8_3">L8.3 Fitting the Master Function to the Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_8_3">L8.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_8_4">L8.4 Principal Component Analysis</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_8_4">L8.4 Exercises</a></td>
    </tr>
</table>



<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L08/NOvA_2020_data_histograms.root, data/L08/NOvA_2020_data_release_predictions_with_systs_all_hists.root <br>
>source: https://publicdocs.fnal.gov/cgi-bin/ShowDocument?docid=17, https://doi.org/10.1103/PhysRevD.106.032004 <br>
>attribution: M. A. Acero et al. (The NOvA Collaboration), Phys. Rev. D 106, 032004 <br>
>license type: Public document

In [ ]:
#>>>RUN: L8.0-runcell00

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L08' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L8.0-runcell01

# for Colab users
!pip install lmfit
!pip install uproot
!pip install astroML

In [ ]:
#>>>RUN: L8.0-runcell02

import numpy as np                 #https://numpy.org/doc/stable/
from scipy import optimize as opt  #https://docs.scipy.org/doc/scipy/reference/optimize.html
import matplotlib.pyplot as plt    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import lmfit                       #https://lmfit.github.io/lmfit-py/ 
import scipy.stats as stats        #https://docs.scipy.org/doc/scipy/reference/stats.html
import uproot                      #https://uproot.readthedocs.io/en/latest/
from sklearn.decomposition import PCA                   #https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
from sklearn.datasets import fetch_lfw_people           #https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html
from sklearn.decomposition import PCA as RandomizedPCA  
from astroML.datasets import sdss_corrected_spectra

In [ ]:
#>>>RUN: L8.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_8_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.1 Neutrino Oscillations</h2>  

| [Top](#section_8_0) | [Previous Section](#section_8_0) | [Exercises](#exercises_8_1) | [Next Section](#section_8_2) |


In [ ]:
#>>>RUN: L8.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L08/slides_L08_01.html', width=970, height=550)

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-8.1.1</span>

For a neutrino beam that has a detector 1000km away, like is present with the DUNE neutrino experiment, what is the optimal energy (in GeV) to observe muon neutrino disappearance given a neutrino energy > 0.5 GeV?

Use the master formula in the code cell below, which outputs the probability of oscillating from a muon neutrino to a muon neutrino, as a function of energy (in units of GeV). Find the minimum of the function to determine where the muon neutrino disappears.

Enter your answer as a number with precision 1e-2. 

In [ ]:
#>>>EXERCISE: L8.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def master_formula(E):
    deltam=1*1e-3
    L=1000
    sin2theta23=0.57
    xval=1.27*deltam*scale1*L/E
    #val=1-4*scale2*sin2theta23*(1-scale2*sin2theta23)*np.sin(xval)**2
    val=1-4*sin2theta23*(1-sin2theta23)*(np.sin(xval)**2)
    return val



<a name='section_8_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.2 Loading the Data</h2>  

| [Top](#section_8_0) | [Previous Section](#section_8_1) | [Exercises](#exercises_8_2) | [Next Section](#section_8_3) |


In [ ]:
#>>>RUN: L8.2-runcell01

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import uproot

file = uproot.open("data/L08/NOvA_2020_data_histograms.root")

#print(file.classnames())

def plot(iLabel,iFile,iColor):
    bin_edges = iFile[iLabel].axis().edges()
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    plt.xlabel("E (GeV)")
    plt.ylabel("$N_{events}$")
    plt.errorbar(bin_centers,iFile[iLabel].values(),yerr=iFile[iLabel].errors(),marker='.',linestyle = '', color = iColor,label=iLabel)    
    
plot("neutrino_mode_numu_quartile1",file,'black')
plot("antineutrino_mode_numu_quartile1",file,'red')
plt.legend()
plt.show()

    

In [ ]:
#>>>RUN: L8.2-runcell02

filePred = uproot.open("data/L08/NOvA_2020_data_release_predictions_with_systs_all_hists.root")

#print(filePred.classnames())
plot("prediction_components_numu_fhc_Quartile1/NoOscillations_Total_pred",filePred,'black')
plot("prediction_components_numu_rhc_Quartile1/NoOscillations_Total_pred",filePred,'red')
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L8.2-runcell03

nquartiles=4
label="neutrino_mode_numu_quartile"
predlabel0="prediction_components_numu_fhc_Quartile"
predlabel1="/NoOscillations_Total_pred"
bin_edges=file[label+"1"].axis().edges()
x = 0.5*(bin_edges[1:] + bin_edges[:-1])
def ratio(iQuartile,iPlot=False):
    ytop=file[label+str(i0+1)].values()
    ytop_err=file[label+str(i0+1)].values()
    ybot=filePred[predlabel0+str(i0+1)+predlabel1].values()
    #ybot_err=file[label+str(i0+1)].values() we will skip this since the error is much smaller
    y = ytop/ybot
    y_err = ytop_err/ybot #we will ignore the ybot error since it is tiny
    if iPlot:
        plt.errorbar(x,y,yerr=y_err,marker='.',linestyle = '',label="Quartile "+str(i0+1))
    return y,y_err
    
for i0 in range(nquartiles):
    ratio(i0,True)
    
plt.xlabel("E(GeV)")
plt.ylabel("Ratio")
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L8.2-runcell04

def combinedRatio():
    y,y_err = ratio(0,False)
    y_arrs=np.array([y])
    weight_arrs=np.array([y_err])
    for i0 in range(nquartiles-1):
        y,y_err = ratio(i0,False)
        y_arrs=np.vstack([y_arrs,y])
        weights=1./(y_err**2)
        weights[weights == np.inf] = 0.1
        weight_arrs = np.vstack([weight_arrs,weights])
    #Now do the weighted 
    yout=np.average(y_arrs,weights=weight_arrs,axis=0)
    weights=np.sum(weight_arrs,axis=0)
    return yout,1/weights**0.5,weights**0.5

label="neutrino_mode_numu_quartile"
predlabel0="prediction_components_numu_fhc_Quartile"
y,yerr,weights=combinedRatio()    

label="antineutrino_mode_numu_quartile"
predlabel0="prediction_components_numu_rhc_Quartile"
y_anti,yerr_anti,weights_anti=combinedRatio()    

plt.errorbar(x,y,yerr=yerr,marker='.',linestyle = '',label="neutrino")
plt.errorbar(x,y_anti,yerr=yerr_anti,marker='.',linestyle = '',label="anti-neutrino")
plt.xlabel("E(GeV)")
plt.ylabel("Ratio")
plt.legend()
plt.ylim(0,1.5)
plt.show()

<a name='exercises_8_2'></a>     

| [Top](#section_8_0) | [Restart Section](#section_8_2) | [Next Section](#section_8_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-8.2.1</span>

In this problem, we want to show that the weighted average with weight given by $\frac{1}{\sigma^2}$ minmizes the uncertainty. For this case, consider a weighted average of two numbers $x$ and $y$. We can define the weighted average $\bar{x}$ as 

$$\bar{x}=f x + (1-f) y$$

where $0 \geq f \geq 1$ is our weight factor. The uncertainty on $\bar{x}$ can be written as 


$$\sigma^2_{\bar{x}}=f^2 \sigma_{x}^2 + (1-f)^2 \sigma_{y}^2$$ 

To minimize the uncertainty all we need to do is 

$$ \frac{d\sigma_{\bar{x}}^2}{df} = 0$$

What is the value of $f$ that minimizes the uncertainty? Express your answer using `sigmax` for $\sigma_{x}$ and `sigmay` for $\sigma_{y}$.


<a name='section_8_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.3 Fitting the Master Function to the Data</h2>  

| [Top](#section_8_0) | [Previous Section](#section_8_2) | [Exercises](#exercises_8_3) | [Next Section](#section_8_4) |


In [ ]:
#>>>RUN: L8.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L08/slides_L08_03.html', width=970, height=550)

In [ ]:
#>>>RUN: L8.3-runcell01

import lmfit
deltam=1*1e-3
L=810
sin2theta23=1.0
def func(x,scale1,scale2):
    xval=1.27*deltam*scale1*L/x
    #val=1-4*scale2*sin2theta23*(1-scale2*sin2theta23)*np.sin(xval)**2
    val=1-4*scale2*(1-scale2)*(np.sin(xval)**2)
    return val

def fit(iX,iY,iWeight):
    model  = lmfit.Model(func)
    p = model.make_params(scale1=1.0,scale2=0.6)
    result = model.fit(x=iX[iY > 0],data=iY[iY > 0], params=p, weights=iWeight[iY > 0])
    lmfit.report_fit(result)
    result.plot()
    print("Fit1 chi2 probability: ",stats.chi2.cdf(result.chisqr,result.nfree))

fit(x,y,weights)
fit(x,y_anti,weights_anti)

In [ ]:
#>>>RUN: L8.3-runcell02

def twoLogLike(var,iX=x,iY=y,iWeights=weights):
    lTot=0
    xtest=func(iX,var[1],var[0])
    lTot = weights*(iY-xtest)
    return np.sum(lTot**2)

from scipy import optimize as opt
x0 = np.array([1,1])
sol=opt.minimize(twoLogLike, x0)

def plotScan(sol):
    #Look the same answers, now let's plot the chi2
    xscan = np.linspace(sol.x[0]*0.6,sol.x[0]*2.5, 100)
    yscan = np.linspace(sol.x[1]*0.6,sol.x[1]*2.0, 100)
    X, Y = np.meshgrid(xscan, yscan)
    levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
    for i0 in range(len(levels)):
        levels[i0] = levels[i0]+sol.fun
    Z = np.array([twoLogLike([xscan,yscan]) for (xscan,yscan) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    fig, ax = plt.subplots(1, 1)
    c = ax.pcolor(X,Y,Z,cmap='RdBu')
    fig.colorbar(c, ax=ax)
    c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
    plt.xlabel("$\sin^{2}\Theta_{23}$")
    plt.ylabel("$\Delta m^{2}_{23}$")
    plt.show()

plotScan(sol)

In [ ]:
#>>>RUN: L8.3-runcell03

#Now let's fix one parameter at the minimum, and profile the other
def scanAxes(sol):
    xscan = np.linspace(sol.x[0]*0.8,sol.x[0]*2.2, 100)
    yscan = np.linspace(sol.x[1]*0.8,sol.x[1]*1.2, 100)

    xLog = np.array([])
    for pX in xscan:
        xLog = np.append(xLog,twoLogLike(var=[pX,sol.x[1]]))

    yLog = np.array([])
    for pY in yscan:
        yLog = np.append(yLog,twoLogLike(var=[sol.x[0],pY]))

    plt.plot(xscan, xLog,label='loglike');
    plt.axhline(sol.fun+1, c='red')
    plt.xlabel("$\sin^{2}\Theta_{23}$")
    plt.ylabel("2$\Delta$LL")
    plt.show()

    #Now for the other parameter
    plt.plot(yscan,yLog,label='LL');
    plt.axhline(sol.fun+1, c='red')
    plt.xlabel("$\Delta m^{2}_{23}$")
    plt.ylabel("2$\Delta$LL")
    plt.show()
    
scanAxes(sol)

In [ ]:
#>>>RUN: L8.3-runcell04

#answer
def twoLogLike(var,iX=x,iY=y_anti,iWeights=weights_anti):
    lTot=0
    xtest=func(iX,var[1],var[0])
    lTot = weights*(iY-xtest)
    return np.sum(lTot**2)

x0 = np.array([1,1])
sol=opt.minimize(twoLogLike, x0)
plotScan(sol)
scanAxes(sol)

In [ ]:
#>>>RUN: L8.3-runcell05

#Now what if we try to add the world's measurement of these parameters into our fit
#https://pdg.lbl.gov/2020/listings/rpp2020-list-neutrino-mixing.pdf
def twoLogLike(var,iX=x,iY=y,iWeights=weights):
    lTot=0
    xtest=func(iX,var[1],var[0])
    lTot = weights*(iY-xtest)
    lTot = np.sum(lTot**2)
    sin2worldavg=0.547
    sin2uncavg=0.021
    constraintsin2=((var[0]-sin2worldavg)**2)/(sin2uncavg**2)
    deltamworldavg=2.453
    deltamuncavg=0.034
    constraintdeltam=((var[1]-deltamworldavg)**2)/(deltamuncavg**2)
    return lTot+constraintsin2+constraintdeltam

x0 = np.array([1,1])
sol=opt.minimize(twoLogLike, x0)
plotScan(sol)
scanAxes(sol)

In [ ]:
#>>>RUN: L8.3-runcell06

def scanAxes(sol):
    xscan = np.linspace(sol.x[0]*0.9,sol.x[0]*1.2, 100)
    yscan = np.linspace(sol.x[1]*0.9,sol.x[1]*1.1, 100)

    xLog = np.array([])
    for pX in xscan:
        xLog = np.append(xLog,twoLogLike(var=[pX,sol.x[1]]))

    yLog = np.array([])
    for pY in yscan:
        yLog = np.append(yLog,twoLogLike(var=[sol.x[0],pY]))

    plt.plot(xscan, xLog,label='loglike');
    plt.axhline(sol.fun+1, c='red')
    plt.xlabel("$\sin^{2}\Theta_{23}$")
    plt.ylabel("2$\Delta$LL")
    plt.show()

    #Now for the other parameter
    plt.plot(yscan,yLog,label='LL');
    plt.axhline(sol.fun+1, c='red')
    plt.xlabel("$\Delta m^{2}_{23}$")
    plt.ylabel("2$\Delta$LL")
    plt.show()
    
scanAxes(sol)

<a name='exercises_8_3'></a>     

| [Top](#section_8_0) | [Restart Section](#section_8_3) | [Next Section](#section_8_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-8.3.1</span>

Looking at the world's best fit values, does the NOvA data improve the best fit and, if so, by how much? Does this improvement make sense? Choose from the options below (optionally draft a computational solution):

- The uncertainty gets better in m23 and sin2theta23. 
- The uncertainty gets better in m23, but worse in sin2theta23. 
- The uncertainty gets worse in m23, but better in sin2theta23. 
- The uncertainty gets worse in m23 and sin2theta23. 

In [ ]:
#>>>EXERCISE: L8.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


<a name='section_8_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L8.4 Principal Component Analysis</h2>     

| [Top](#section_8_0) | [Previous Section](#section_8_3) | [Exercises](#exercises_8_4) |


In [ ]:
#>>>RUN: L8.4-runcell01
from sklearn.decomposition import PCA
import numpy.linalg as la
#make some toy data
lAs = np.random.normal(0,1,1000)
lBs = np.random.normal(0,1,1000)+0.5*lAs
cov = np.cov([lAs,lBs])
#eigen cov
w, v=la.eig(cov)


plt.plot(lAs,lBs,".")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

X=(np.vstack([lAs,lBs])).T
pca = PCA(n_components=2)
pca.fit(X)
print("PCA vectors")
print(pca.components_)
print("PCA values")
print(pca.explained_variance_)
print("Old Eigen","vectors",w,"values",v)

def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',linewidth=2,shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(lAs, lBs, alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

In [ ]:
#>>>RUN: L8.4-runcell02

#Now let's do it ML style for fun
#Load some images of faces
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)

fig, axes = plt.subplots(3, 8, figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
#Let's plot the eigenvectors
for i, ax in enumerate(axes.flat):
    ax.imshow(faces.data[i].reshape(62, 47), cmap='bone')
    
#Fit them to PCA 
from sklearn.decomposition import PCA as RandomizedPCA
pca = RandomizedPCA(200)
pca.fit(faces.data)
fig, axes = plt.subplots(3, 8, figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
#Let's plot the eigenvectors
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')
plt.show()
    
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()


In [ ]:
#>>>RUN: L8.4-runcell03

# Compute the components and projected faces
pca = RandomizedPCA(80).fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),subplot_kw={'xticks':[], 'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')
    
ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('80-dim\nreconstruction');


In [ ]:
#>>>RUN: L8.4-runcell04

from astroML.datasets import sdss_corrected_spectra

data = sdss_corrected_spectra.fetch_sdss_corrected_spectra()
wavelengths = sdss_corrected_spectra.compute_wavelengths(data)
spectra_raw = data['spectra']
count=-1
#let's plot this guy
fig, ax = plt.subplots(2, 10, figsize=(20, 5.5),subplot_kw={'yticks':[]},gridspec_kw=dict(hspace=0.1, wspace=0.1))
nrows = 2; ncols = 10
for i in range(ncols):
    for j in range(nrows):
        count=count+1
        ax[j,i].plot(wavelengths,spectra_raw[count], '-k', lw=1)
        ax[j,i].set_xlim(3100, 7999)
        if j < nrows - 1:
            ax[j,i].xaxis.set_major_formatter(plt.NullFormatter())
        else:
            ax[j,i].set_xlabel(r'wavelength $(\AA)$')
            
plt.show()


<a name='exercises_8_4'></a>   

| [Top](#section_8_0) | [Restart Section](#section_8_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-8.4.1</span>

Your task is to run PCA, look at the dominant eigenvector, and find the top two spectral lines. What do they correspond to (refer to the table <a href="http://astronomy.nmsu.edu/drewski/tableofemissionlines.html" target="_blank">here</a>.)?

Select two spectral lines from the options below (in units of angstroms):

- He ~4200
- H$\gamma$ ~4350
- H$\beta$ ~4860
- O III ~4950
- O III ~5006
- H$\alpha$ ~6560
- N II ~6580
- O I ~7000

Does this make sense?


In [ ]:
#>>>EXERCISE: L8.4.1

# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.
from astroML.datasets import sdss_corrected_spectra

data = sdss_corrected_spectra.fetch_sdss_corrected_spectra()
wavelengths = sdss_corrected_spectra.compute_wavelengths(data)
spectra_raw = data['spectra']

#select only the first 850 bins to avoid noise features
spectra_raw = spectra_raw[:,0:850]
wavelengths = wavelengths[0:850]
pca = RandomizedPCA()


#YOUR CODE HERE
#fit the pca to the spectra_raw data 
#plot the first eigenvector, pca.components_[0] over the full range of wavelengths
#optionally constrain the wavelength range to zoom in on spectral lines

>#### Follow-up 8.4.1a (ungraded)
>
>Plot all of the eigenvectors. What are the dominant features of each?